In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *
from setting import *

In [ ]:
target_x_sample = pd.read_table(
    PATH_DICT['target_x_sample_file_path'],
    index_col=0,
)

In [ ]:
groups = {
    'Group 0': {
        'gene_sets': GENE_SET_TO_PEEK,
        'size': 16,
        'color': '#20d9ba',
    },
}

In [ ]:
for target_name in target_x_sample.index:
    
    output_directory_path = '../output/hill_plot/{}'.format(target_name)
    
    ccal.establish_path(
        output_directory_path,
        'directory',
    )
    
    score_moe_p_value_fdr = pd.read_table(
        '{}/{}/all.tsv'.format(
            PATH_DICT['differentially_expressed_gene_set_directory_path'],
            target_name,
        ),
        index_col=0,
    )
    
    for gene_set_file_path in PATH_DICT['gene_set_file_paths']:
        
        gene_set_file_name = gene_set_file_path.split('/')[-1]
        
        gmt = ccal.read_gmt(gene_set_file_path)
        
        score_moe_p_value_fdr__gmt = score_moe_p_value_fdr.loc[gmt.index].sort_values('Score')
         
        xs = [1 - score_moe_p_value_fdr__gmt['P-Value']]

        ys = [score_moe_p_value_fdr__gmt['Score']]

        names = [gene_set_file_name]

        markers =[dict(
            color='#d0d0d0',
            size=3.2,
        )]

        texts = [gmt.index]

        for group_name, group in groups.items():

            group_gene_sets = group['gene_sets']

            score_moe_p_value_fdr__gmt__peek = score_moe_p_value_fdr__gmt.loc[[gene_set in group_gene_sets for gene_set in score_moe_p_value_fdr__gmt.index]]

            xs.append(1 - score_moe_p_value_fdr__gmt__peek['P-Value'])

            ys.append(score_moe_p_value_fdr__gmt__peek['Score'])

            names.append(group_name)

            texts.append(score_moe_p_value_fdr__gmt__peek.index)

            markers.append(dict(
                size=group['size'],
                color=group['color'],
            ))

        ccal.plot_points(
            xs,
            ys,
            names=names,
            modes=(
                'lines',
                'markers',
            ),
            texts=texts,
            markers=markers,
            title='{}<br>{}'.format(
                target_name,
                gene_set_file_name,
            ),
            xaxis_title='1 - P-Value',
            yaxis_title='Gene Set Score',
            html_file_path='{}/{}'.format(
                output_directory_path,
                gene_set_file_name,
            ),
        )